# Qulacs Classes

## QuantumStateクラス
complex128の精度で\f$2^n\f$個の複素配列をCPU/GPU上に確保し管理するクラスです。状態ベクトルの移動や変換の他、状態に関する情報の計算や変形をサポートします。

### 生成と破棄
インスタンス生成時に必要なメモリが確保されます.

In [1]:
from qulacs import QuantumState
n = 2
state = QuantumState(n)

<code>\_\_repr\_\_</code>関数のオーバーライドにより状態ベクトルのフォーマットされた表示を提供します。

In [2]:
print(state)

*** Quantum State ***
 * Qubit Count : 2
 * Dimension   : 4
 * State vector : 
(1,0)
(0,0)
(0,0)
(0,0)



メモリはpythonがインスタンスを破棄した段階で解放されますが、メモリ解放のために明示的に破棄したい場合は<code>del</code>で解放できます。

In [3]:
del state

### 量子状態とnumpy array間の変換
また、<code>get_vector</code>および<code>load</code>関数で量子状態とnumpy arrayの相互変換が可能です。ノルムが保存しているかなどは原則チェックされません。

In [4]:
from qulacs import QuantumState

state = QuantumState(2)
vec = state.get_vector()
print(vec)
state.load([0,1,2,3])
print(state.get_vector())


[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 1.+0.j 2.+0.j 3.+0.j]


### 量子状態間のコピー
量子状態は<code>copy</code>で自身と同じインスタンスを新たに生成できます。また<code>load</code>関数に量子状態を与えることで、既存の量子状態に新たに領域を確保することなく別の量子状態の量子ベクトルをコピーすることが出来ます。これにより既に確保した領域を再利用できます。既に持っている量子状態と同じサイズの状態ベクトルを確保して、状態のコピーはしなくてよい場合は<code>allocate_buffer</code>関数を使えます。

In [5]:
from qulacs import QuantumState

initial_state = QuantumState(3)
buffer = initial_state.allocate_buffer()
for ind in range(10):
	buffer.load(initial_state)
	# some computation and get results

### 量子状態の初期化
下記は量子状態を特定の状態に初期化する関数です。

In [6]:
from qulacs import QuantumState

n = 3
state = QuantumState(n)

In [7]:
# |0>状態へ初期化
state.set_zero_state()
print(state.get_vector())

# 指定値の二進数表記の計算基底へ初期化
state.set_computational_basis(0b101)
print(state.get_vector())

# 引数の値をシードとしてハール測度でランダムな純粋状態へ初期化
# 指定値が無い場合はtime関数がシードとして使われる。疑似乱数はxorshiftを利用。
state.set_Haar_random_state(0)
print(state.get_vector())
``

SyntaxError: invalid syntax (<ipython-input-7-2e89d96d89f2>, line 13)

### 量子状態の検査
下記は量子状態を変えずに量子状態の情報を調べる関数の一覧です。

In [ ]:
from qulacs import QuantumState

n = 5
state = QuantumState(n)
state.set_Haar_random_state(0)

# 量子ビットの数を得る。
qubit_count = state.get_qubit_count()
print("qubit_count", qubit_count)

# 指定番目の量子ビットが0に測定される確率を得る
prob = state.get_zero_probability(1)
print("zero_prob_1", prob)

# 任意の周辺確率を得る
# 引数は量子ビット数と同じ長さの配列
# 0,1,2を指定する。0,1はその添え字がその値で測定される確率、
# 2はそのビットを周辺化することを意味する。
# 例えば、3番目が0で、0番目が1と測定される確率の計算は下記
prob = state.get_marginal_probability([1,2,2,0,2])
print("marginal_prob", prob)

# Z基底で測定した時の確率分布のエントロピーを得る
ent = state.get_entropy()
print("entropy", ent)

# squared norm (<a|a>)の取得
# Trace preservingでない操作が可能なため、状態のノルムが1とは限らない
sq_norm = state.get_squared_norm()
print("sqaured_norm", sq_norm)

# 引数で与えた数の回数Z基底で全量子ビットを測定しサンプリングする。
# 得られるバイナリを整数値にしたもののリストを得る。
samples = state.sampling(10)
print("sampling", samples)

# 状態ベクトルがCPU/GPUのどちらにあるかを文字列で取得する
dev_type = state.get_device_name()
print("device", dev_type)

qubit_count 5
zero_prob_1 0.5151171741097239
marginal_prob 0.40401228349316776
entropy 3.0788233113265715
sqaured_norm 1.0
sampling [23, 9, 8, 4, 25, 7, 7, 24, 20, 27]
device cpu

### 量子状態の変形

下記の関数は量子状態を書き換える関数です。

In [ ]:
from qulacs import QuantumState
state = QuantumState(2)
state.set_computational_basis(0)
buffer = QuantumState(2)
buffer.set_computational_basis(2)
print("state" , state.get_vector())
print("buffer", buffer.get_vector())

# 量子状態間の和(state <- state+buffer)
# stateにbufferの状態を足し重ね合わせ状態を作ります。 
# 操作後のノルムは一般に1ではありません。
state.add_state(buffer)
print("added", state.get_vector())

# 量子状態と複素数の積
# 引数の複素数を全要素に掛けます。
# 操作後のノルムは一般に1ではありません。
coef = 0.5 + 0.1j
state.multiply_coef(coef)
print("mul_coef", state.get_vector())

# 量子状態の正規化
# 引数として現在のsquared normを与える必要があります。
squared_norm = state.get_squared_norm()
print("sq_norm", squared_norm)
state.normalize(squared_norm)
print("normalized", state.get_vector())
print("sq_norm", state.get_squared_norm())

### 古典レジスタの操作

量子状態は可変長の整数配列を古典レジスタを持ちます。古典レジスタはInstrument操作の結果を書き込んだり、古典レジスタの結果を条件として実行するゲートを記述するのに用います。まだ書き込まれていない古典レジスタの値は0です。なお、古典レジスタは<code>copy,load</code>関数で量子状態を複製した際に同時に複製されます。

In [ ]:
from qulacs import QuantumState
state = QuantumState(3)
position = 0
# position番目にvalueを書き込みます
value = 20
state.set_classical_value(position, value)
# position番目のレジスタ値を得ます
obtained = state.get_classical_value(position)
print(obtained)

### 量子状態間の計算
量子状態間の内積は<code>inner_product</code>で得られます。

In [ ]:
from qulacs import QuantumState
from qulacs.state import inner_product

n = 5
state_bra = QuantumState(n)
state_ket = QuantumState(n)
state_bra.set_Haar_random_state()
state_ket.set_computational_basis(0)

# 内積値の計算
value = inner_product(state_bra, state_ket)
print(value)

### GPUを用いた計算

Qulacsをqulacs-gpuパッケージからインストールした場合、<code>QuantumStateGpu</code>クラスが使用できます。クラス名が異なる以外、利用方法は<code>QuantumState</code>と同じです。

In [ ]:
from qulacs import QuantumStateGpu
state = QuantumStateGpu(2)
print(state)
print(state.get_device_name())

In [ ]:
gpu
使い方は<code>QuantumState</code>と同様ですが、二点留意点があります。
1. <code>get_vector</code>関数はGPU/CPU間のコピーを要するため長い時間がかかります。出来る限りこの関数の利用を回避して計算を行うべきです。
2. CPU/GPUの状態間の<code>inner_product</code>は計算できません。GPUとCPUの状態ベクトルの間で状態ベクトルの<code>load</code>を行うことは可能ですが、時間がかかるので避けるべきです。